In [322]:
import pandas as pd
import numpy as np
import re
from rapidfuzz import fuzz
import string
from googletrans import Translator # version: pip install googletrans==3.1.0a0

In [323]:
data =pd.read_excel('Ozurgeti.xlsx')                   

In [324]:
data.head(4)
data.columns

Index(['საიდენტიფიკაციო ნომერი', 'პირადი ნომერი',
       'ორგანიზაციულ-სამართლებრივი ფორმა', 'სუბიექტის დასახელება',
       'რეგიონი (იურიდიული)', 'მუნიციპალიტეტი/ქალაქი (იურიდიული)',
       'დაბის/თემის/სოფლის საკრებულო (იურიდიული)', 'დაბა/სოფელი (იურიდიული)',
       'იურიდიული მისამართი', 'რეგიონი (ფაქტობრივი)',
       'მუნიციპალიტეტი/ქალაქი (ფაქტობრივი)',
       'დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი)', 'დაბა/სოფელი (ფაქტობრივი)',
       'ფაქტობრივი მისამართი', 'საქმიანობის კოდი NACE Rev.2',
       'საქმიანობის დასახელება NACE Rev.2', 'პარტნიორი', 'ხელმძღვანელი',
       'ტელეფონი', 'ელფოსტა', 'საკუთრების ფორმა',
       'აქტიური ეკონომიკური სუბიექტები', 'ბიზნესის ზომა'],
      dtype='object')

In [325]:
data.head(3)

,საიდენტიფიკაციო ნომერი,პირადი ნომერი,ორგანიზაციულ-სამართლებრივი ფორმა,სუბიექტის დასახელება,რეგიონი (იურიდიული),მუნიციპალიტეტი/ქალაქი (იურიდიული),დაბის/თემის/სოფლის საკრებულო (იურიდიული),დაბა/სოფელი (იურიდიული),იურიდიული მისამართი,რეგიონი (ფაქტობრივი),...,ფაქტობრივი მისამართი,საქმიანობის კოდი NACE Rev.2,საქმიანობის დასახელება NACE Rev.2,პარტნიორი,ხელმძღვანელი,ტელეფონი,ელფოსტა,საკუთრების ფორმა,აქტიური ეკონომიკური სუბიექტები,ბიზნესის ზომა
0,100015680,6.200100e+10,იმ,ზურაბ შელეგია,ქ. თბილისი,ნაძალადევის რაიონი,NaN,NaN,ქ. თბილისის ნაძალადევის რაიონში / მანჯგალაძის 29,გურია,...,ოზურგეთის რაიონში/დაბ. ურეკიƒ.“დდ„დ‰დˆ,47.11.0,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",ზურაბ შელეგია - 100,ზურაბ შელეგია,NaN,NaN,კერძო ადგილობრივი საკუთრება,NaN,NaN
1,100019383,6.200100e+10,იმ,მარინე ჭელიძე,ქ. თბილისი,ნაძალადევის რაიონი,NaN,NaN,"ქ. თბილისის ნაძალადევის რაიონში / სანზონა კ.6,ბ.1",გურია,...,NaN,47.11.0,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",მარინე ჭელიძე - 100,მარინე ჭელიძე,NaN,NaN,კერძო ადგილობრივი საკუთრება,NaN,NaN
2,100085300,3.300100e+10,იმ,დავით ანთაძე,გურია,ოზურგეთის მუნიციპალიტეტი,ბახვის თემის საკრებულო,NaN,ბახვის თემის საკრებულო,გურია,...,ბახვის თემის საკრებულო,NaN,NaN,დავით ანთაძე - 100,დავით ანთაძე,NaN,NaN,კერძო ადგილობრივი საკუთრება,NaN,NaN


In [ ]:
data = data[['საიდენტიფიკაციო ნომერი',
             'პირადი ნომერი',
             'დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი)',
             'ორგანიზაციულ-სამართლებრივი ფორმა',
             'სუბიექტის დასახელება','ფაქტობრივი მისამართი',
             'საქმიანობის დასახელება NACE Rev.2',
             'საქმიანობის კოდი NACE Rev.2',
             'აქტიური ეკონომიკური სუბიექტები',
             'ბიზნესის ზომა']]

In [327]:
data.head(4)

,საიდენტიფიკაციო ნომერი,პირადი ნომერი,დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი),ორგანიზაციულ-სამართლებრივი ფორმა,სუბიექტის დასახელება,ფაქტობრივი მისამართი,საქმიანობის დასახელება NACE Rev.2,აქტიური ეკონომიკური სუბიექტები,ბიზნესის ზომა
0,100015680,6.200100e+10,NaN,იმ,ზურაბ შელეგია,ოზურგეთის რაიონში/დაბ. ურეკიƒ.“დდ„დ‰დˆ,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",NaN,NaN
1,100019383,6.200100e+10,ურეკის სადაბო საკრებულო,იმ,მარინე ჭელიძე,NaN,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",NaN,NaN
2,100085300,3.300100e+10,ბახვის თემის საკრებულო,იმ,დავით ანთაძე,ბახვის თემის საკრებულო,NaN,NaN,NaN
3,100096389,3.300101e+10,NaN,იმ,სოსო სადრაძე,შემოქმედის,"სხვა ხილის, კენკროვნების და კაკლისებრების მოყვანა",NaN,NaN


In [328]:

data = data[(data['დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი)'].isna()) |(data['დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი)'].str.contains('ქ. ოზურგეთი'))]


In [329]:
data = data[~(data['ფაქტობრივი მისამართი'].isnull())]

In [330]:
data

,საიდენტიფიკაციო ნომერი,პირადი ნომერი,დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი),ორგანიზაციულ-სამართლებრივი ფორმა,სუბიექტის დასახელება,ფაქტობრივი მისამართი,საქმიანობის დასახელება NACE Rev.2,აქტიური ეკონომიკური სუბიექტები,ბიზნესის ზომა
0,100015680,6.200100e+10,NaN,იმ,ზურაბ შელეგია,ოზურგეთის რაიონში/დაბ. ურეკიƒ.“დდ„დ‰დˆ,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",NaN,NaN
3,100096389,3.300101e+10,NaN,იმ,სოსო სადრაძე,შემოქმედის,"სხვა ხილის, კენკროვნების და კაკლისებრების მოყვანა",NaN,NaN
4,100106966,1.003006e+09,NaN,იმ,ზაალი კუჭავა,"ოზურგეთი, სოფ.მაყვანეთი.",ღორღიანი და ქვიშოვანი კარიერების დამუშავება,NaN,NaN
5,100113949,1.013002e+09,NaN,იმ,ნინო თევდორაშვილი,ოზურგეთის რაიონში/დაბ. ურეკი დ. ურეკი,NaN,NaN,NaN
6,100118169,1.017015e+09,NaN,იმ,ნინო გიორგაძე,ოზურგეთის რაიონში/დაბ. ურეკი„.დ‡დ€დ—დ€დˆდ˜დ…დˆ...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6892,62006022557,6.200602e+10,NaN,იმ,გიზო ჯალაღონია,გურიის ქუჩა N35,საცხოვრებელი და არასაცხოვრებელი შენობების მშენ...,NaN,NaN
6893,62006026579,6.200603e+10,NaN,იმ,ინდივიდუალური მეწარმე ვალენტინა გოგოხია,შეკვეთილი,სასტუმროები და განთავსების მსგავსი საშუალებები...,აქტიური,მცირე
6894,62007002235,6.200700e+10,NaN,იმ,სპარტაკ კობერიძე,"ქ. ოზურგეთი, ქუჩა ერისთავი, N 2",NaN,NaN,NaN
6896,62011004299,6.201100e+10,NaN,იმ,შოთა მდინარაძე,დაბ. ურეკი თაყაიშვილის 161,NaN,NaN,NaN


In [331]:
def clean_text(arg):
    cleaned = re.sub(r'[^\w\s]',' ',arg)
    return cleaned

def remove_punctuation(input_string):
    result = input_string
    for char in string.punctuation:
        result = result.replace(char, ' ')
    return result


In [332]:
data['ნომერი'] = data['ფაქტობრივი მისამართი'].apply(lambda x: "".join([i for i in x if i.isnumeric()]))

In [333]:
data["ქუჩა"] = data["ფაქტობრივი მისამართი"].apply(clean_text)
data["ქუჩა"] = data["ფაქტობრივი მისამართი"].apply(remove_punctuation)

In [334]:
data['ქუჩა'] = data['ქუჩა'].apply(lambda x: "".join([re.sub('\d+', '',i)  for i in x ]))

In [335]:
data['ქუჩა'] = data['ქუჩა'].apply(lambda x: "".join([re.sub(r'\\N', '',i)  for i in x ]))

In [336]:
data['ქუჩა'] =data['ქუჩა'].apply(lambda x: ''.join([i for i in x if i != 'N']))

In [337]:
data['ქუჩა'] =data['ქუჩა'].apply(lambda x: ' '.join([i for i in x.split() if i not in [ 'ქუჩა',
                                                                                        'საქართველო',
                                                                                        'ოზურგეთი',
                                                                                        'ოზურგეთის',
                                                                                        'რაიონში',
                                                                                        'სახელობის',
                                                                                        'შესახვევი',
                                                                                        'შესახ',
                                                                                        'ჩიხი'
                                                                                        ]
                                                                                    ]
                                                                                )
                                                                            )

In [338]:
data['ქუჩა'] = data['ქუჩა'].apply(lambda x: ' '.join([i for i in x.split() if i.isnumeric() or len(i) >3]))

In [339]:
data['ქუჩა'] = data['ქუჩა'].apply(
    lambda x: re.sub(r'თაყაიშვილ(ი|ის)', 'საჯავახო — ჩოხატაური — ოზურგეთი — ქობულეთი', x) if isinstance(x, str) else x)

In [340]:
data = data[ (data['ქუჩა']!='')]
print(data.shape)

(3630, 11)


In [341]:
data = data[(data['ნომერი']!='')]
print(data.shape)

(2864, 11)


In [342]:
data.sort_values(by='ქუჩა', ascending=True)

,საიდენტიფიკაციო ნომერი,პირადი ნომერი,დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი),ორგანიზაციულ-სამართლებრივი ფორმა,სუბიექტის დასახელება,ფაქტობრივი მისამართი,საქმიანობის დასახელება NACE Rev.2,აქტიური ეკონომიკური სუბიექტები,ბიზნესის ზომა,ნომერი,ქუჩა
5975,33001066269,3.300107e+10,ქ. ოზურგეთი,იმ,დავით ბაბილოძე,აბაშიძის 5,"საცალო ვაჭრობა რკინა-კავეულით, საღებავებით და ...",NaN,NaN,5,აბაშიძის
4724,33001025731,3.300103e+10,ქ. ოზურგეთი,იმ,ცისანა ფრანგულაშვილი,აბაშიძის ქუჩა #6,სხვა საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში,აქტიური,მცირე,6,აბაშიძის
1381,137106049,3.300105e+10,NaN,იმ,ეთერი ვაშაკიძე,აბაშიძის 2,NaN,NaN,NaN,2,აბაშიძის
6470,33001079313,3.300108e+10,ქ. ოზურგეთი,იმ,ლევანი ბერიძე,აბაშიძის ქუჩა #9,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",აქტიური,მცირე,9,აბაშიძის
6818,61004034884,6.100403e+10,NaN,იმ,ცირა ართმელაძე,აკაკი წერეთლის №26,"საცალო ვაჭრობა რკინა-კავეულით, საღებავებით და ...",აქტიური,მცირე,26,აკაკი წერეთლის
...,...,...,...,...,...,...,...,...,...,...,...
1611,137109698,3.300104e+10,NaN,იმ,რუბენ გუბელიძე,ჯორჯიაშვილის გ შესსახლი.1,NaN,NaN,NaN,1,ჯორჯიაშვილის შესსახლი
1574,137108966,3.300102e+10,NaN,იმ,იური ლომინაძე,ჯორჯიაშვილის გ შესსახლი.1,NaN,NaN,NaN,1,ჯორჯიაშვილის შესსახლი
3393,33001000920,3.300100e+10,NaN,იმ,ზაზა საჯაია,ჯორჯიაშვილისI ჩიხი N19,NaN,NaN,NaN,19,ჯორჯიაშვილისI
6578,35001031935,3.500103e+10,NaN,იმ,ქეთევან მეფარიშვილი,ჯორჯიაშვილისN12ბ 31,NaN,NaN,NaN,1231,ჯორჯიაშვილისბ


In [343]:
def translate_text(text, dest_language='en'): 
    translator = Translator()
    result = translator.translate(text, dest=dest_language)
    return result.text

In [344]:
data.head(3)

,საიდენტიფიკაციო ნომერი,პირადი ნომერი,დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი),ორგანიზაციულ-სამართლებრივი ფორმა,სუბიექტის დასახელება,ფაქტობრივი მისამართი,საქმიანობის დასახელება NACE Rev.2,აქტიური ეკონომიკური სუბიექტები,ბიზნესის ზომა,ნომერი,ქუჩა
6,100118169,1.017015e+09,NaN,იმ,ნინო გიორგაძე,ოზურგეთის რაიონში/დაბ. ურეკი„.დ‡დ€დ—დ€დˆდ˜დ…დˆ...,NaN,NaN,NaN,51,ურეკი„ დ‡დ€დ—დ€დˆდ˜დ…დˆდŠდˆდ‘
7,100128238,3.300101e+10,NaN,იმ,ლევან ყარგიშვილი,გრიბოედოვის ქ31 ბ32,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",აქტიური,მცირე,3132,გრიბოედოვის
8,100129488,1.023003e+09,NaN,იმ,გრიგოლ დუმბაძე,დაბ. ურეკი თაყაიშვილის 152,სასტუმროები და განთავსების მსგავსი საშუალებები...,NaN,NaN,152,ურეკი საჯავახო — ჩოხატაური — ოზურგეთი — ქობულეთის


In [345]:
def fuzzy_match(ops_name, customer_input):
    similarity_score = fuzz.ratio(ops_name, customer_input)
    return similarity_score, customer_input

In [346]:
OPS_Streets = pd.read_csv('Ozurgeti_streets.txt',header =None, names=['street_name'])
OPS_Streets['street_name'] = OPS_Streets['street_name'].apply(lambda x: x.replace('ქუჩა','') if 'ქუჩა' in x.split() else x)
OPS_Streets = OPS_Streets[~OPS_Streets['street_name'].str.contains(r'\b(?:ჩიხი|შესახვევი)\b', regex=True)]

In [348]:
import pandas as pd

def match_ops_street(row):
    if row['დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი)'] == 'ქ. ოზურგეთი':
        for i in OPS_Streets['street_name']:
            if isinstance(row['ქუჩა'], str) and row['ქუჩა'].strip() in i.split() and len(row['ქუჩა'].strip()) <= len(i.strip()):
                return i
        return row['ქუჩა']
    
    elif pd.isna(row['დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი)']):
        for i in OPS_Streets['street_name']:
            if isinstance(row['ქუჩა'], str) and row['ქუჩა'].strip() in i.split() and len(row['ქუჩა'].strip()) <= len(i.strip()):
                return i
        return None
    

# Apply the function row-wise
data['ქუჩა_OPS'] = data.apply(match_ops_street, axis=1)


In [349]:
data['Similarity'] = data['ქუჩა_OPS'].apply(
    lambda x: max([fuzzy_match(x, street) for street in OPS_Streets['street_name']])
)

In [350]:
data[['similarity_score', 'matched_street']] = pd.DataFrame(data['Similarity'].tolist(), index=data.index)

In [351]:
data.drop(columns='Similarity', inplace=True)

In [352]:
data.head(4)

,საიდენტიფიკაციო ნომერი,პირადი ნომერი,დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი),ორგანიზაციულ-სამართლებრივი ფორმა,სუბიექტის დასახელება,ფაქტობრივი მისამართი,საქმიანობის დასახელება NACE Rev.2,აქტიური ეკონომიკური სუბიექტები,ბიზნესის ზომა,ნომერი,ქუჩა,ქუჩა_OPS,similarity_score,matched_street
6,100118169,1.017015e+09,NaN,იმ,ნინო გიორგაძე,ოზურგეთის რაიონში/დაბ. ურეკი„.დ‡დ€დ—დ€დˆდ˜დ…დˆ...,NaN,NaN,NaN,51,ურეკი„ დ‡დ€დ—დ€დˆდ˜დ…დˆდŠდˆდ‘,None,0.0,ჯორჯიაშვილის
7,100128238,3.300101e+10,NaN,იმ,ლევან ყარგიშვილი,გრიბოედოვის ქ31 ბ32,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",აქტიური,მცირე,3132,გრიბოედოვის,ალექსანდრე გრიბოედოვის,100.0,ალექსანდრე გრიბოედოვის
8,100129488,1.023003e+09,NaN,იმ,გრიგოლ დუმბაძე,დაბ. ურეკი თაყაიშვილის 152,სასტუმროები და განთავსების მსგავსი საშუალებები...,NaN,NaN,152,ურეკი საჯავახო — ჩოხატაური — ოზურგეთი — ქობულეთის,None,0.0,ჯორჯიაშვილის
16,100201069,1.030014e+09,NaN,იმ,რემანოზ ზამთარაძე,დაბ. ურეკი თაყაიშვილის 176,რესტორნები და საკვებით მობილური მომსახურების ს...,აქტიური,მცირე,176,ურეკი საჯავახო — ჩოხატაური — ოზურგეთი — ქობულეთის,None,0.0,ჯორჯიაშვილის


In [353]:
def fill_ops(row):
    if row['similarity_score'] > 90:
        return row['matched_street']
    else:
        return row['ქუჩა_OPS']

data['ქუჩა_საბოლოო'] = data.apply(fill_ops, axis=1)

In [354]:
data = data[~data['ქუჩა_საბოლოო'].isna()]

In [355]:
data.loc[:, 'ქუჩა_საბოლოო'] = data['ქუჩა_საბოლოო'].apply(str.strip)


In [356]:
data.loc[:,'St_Full_Name']=data['ნომერი'] + ' ' + data['ქუჩა_საბოლოო'] + ' ქუჩა, Ozurgeti, Georgia'

In [357]:
data.drop(columns=['დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი)','ფაქტობრივი მისამართი','ნომერი','ქუჩა','ქუჩა_OPS','similarity_score','matched_street','ქუჩა_საბოლოო'])

,საიდენტიფიკაციო ნომერი,პირადი ნომერი,ორგანიზაციულ-სამართლებრივი ფორმა,სუბიექტის დასახელება,საქმიანობის დასახელება NACE Rev.2,აქტიური ეკონომიკური სუბიექტები,ბიზნესის ზომა,St_Full_Name
7,100128238,3.300101e+10,იმ,ლევან ყარგიშვილი,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",აქტიური,მცირე,"3132 ალექსანდრე გრიბოედოვის ქუჩა, Ozurgeti, Ge..."
38,102370195,3.300103e+10,იმ,სოფიო ღლონტი,NaN,NaN,NaN,"1 ჯორჯიაშვილის ქუჩა, Ozurgeti, Georgia"
53,102398219,3.300103e+10,იმ,ელისო ქადარია,სხვა საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში,აქტიური,მცირე,"3 ბათუმის ქუჩა, Ozurgeti, Georgia"
55,102399940,3.300103e+10,იმ,თამარი გიორგაძე,საცალო ვაჭრობა სხვა ნახმარი საქონლით,აქტიური,მცირე,"13 ერისთავის ქუჩა, Ozurgeti, Georgia"
63,104430894,3.300101e+10,იმ,დავით ყანდიაშვილი-აუდიენცია,NaN,NaN,NaN,"24 ნიკოლაძის ქუჩა, Ozurgeti, Georgia"
...,...,...,...,...,...,...,...,...
6872,61010006946,6.101001e+10,იმ,ირა ირემაძე,რესტორნები და საკვებით მობილური მომსახურების ს...,აქტიური,მცირე,"35 გურიის ქუჩა, Ozurgeti, Georgia"
6874,61010016559,6.101002e+10,იმ,ნათელა თოიძე,საცალო ვაჭრობა სხვა საქონლით ფარდულებში და ბაზ...,აქტიური,მცირე,"21 აღმაშენებლის ქუჩა, Ozurgeti, Georgia"
6891,62006018863,6.200602e+10,იმ,ხათუნა ელიავა,საცალო ვაჭრობა თამბაქოს ნაწარმით სპეციალიზებულ...,NaN,NaN,"10 აღმაშენებლის ქუჩა, Ozurgeti, Georgia"
6892,62006022557,6.200602e+10,იმ,გიზო ჯალაღონია,საცხოვრებელი და არასაცხოვრებელი შენობების მშენ...,NaN,NaN,"35 გურიის ქუჩა, Ozurgeti, Georgia"


In [358]:
data.to_excel("Ozurgeti_Steet_WithFuzzy.xlsx", index=False)

________________

** რაც საზღვრის მიღმაა დავიკიდოთ

** ქუჩის პარამეტრები
